## Libraries

In [1]:
#Import All Dependencies
# import cv2, os, bz2, json, csv, difflib, requests, socket, whois, urllib.request, urllib.parse, urllib.error, re, OpenSSL, ssl
import numpy as np
from datetime import datetime
from urllib.parse import urlparse
from urllib.request import Request, urlopen
# from selenium import webdriver
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
# from timeout import timeout
import requests
import numpy as np
import urllib
import cv2
import re
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.options import Options
from PIL import Image
from io import BytesIO
import time
import os
import os.path
from os import path
import io
from difflib import SequenceMatcher
import json

In [2]:
import contextlib 
  
try: 
    from urllib.parse import urlencode           
except ImportError: 
    from urllib import urlencode 
try: 
    from urllib.request import urlopen 
except ImportError: 
    from urllib2 import urlopen 

import sys 

### TinyURL

In [3]:
from TinyURL import getTinyURL

In [4]:
#Returns Beautiful Soup object
def getHTML(URL):
    try:
        hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36'} #Make the user agent verified, that is Mozilla
#         req = Request(URL,headers=hdr)
        req = requests.get(URL, headers=hdr)
        page = req.text #Get URL HTML contents
        soup = BeautifulSoup(page, 'html.parser') #Convert to BeutifulSoup
        print("Built Soup")
        #prettyText = str(soup.prettify()) #Convert the HTML in its form
        return soup
    except Exception as e:
# #         if e.__class__.__name__ == "TimeoutError": raise TimeoutError("")
        return None

### XPath

In [5]:
from UnitTestingOnXPath import getMyStackXPath, getMyXPath, getStackXPath

In [6]:
def getListOfXPath(listOfElements):
    listOfXPath = []
    for element in listOfElements:
        listOfXPath.append(getMyXPath(element))
    return listOfXPath

### Selenium

In [7]:
fireFoxBinaryPath = r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe'
geckoDriverPath = "C:/Users/Yash/Documents/geckodriver.exe"

googleChomreDriverPath = r"C:\Users\Yash\Documents\chromedriver.exe"

windowSizeXAxis = 1024
windowSizeYAxis = 768
headlessOrNot = True

In [8]:
from SeleniumScript import getSeleniumDriverForChrome, takeScreenshotWithChrome

### Similarity

In [9]:
highlySimilarValue = 0.40 #Denotes high chances. If occured, remove the less elements
moderatelySimilarValue = 0.15 #Denotes moderate chances. If occured, remove the less elements
bareMinimumSimilarValue = 0.05 #Denotes elements which have barely any similarity, so discard the elements below them

In [10]:
def similar(a, b):
    a = a.strip()
    b = b.strip()
    return SequenceMatcher(None, a, b).ratio()

### Element Extractor

1. Try different varaitions of price text.
2. Check if the size of the element is more than 1x1 //Walmart Page cause problem using hidden element
3. 

In [11]:
#Returns the elements for sepcified regex which will match the tag content/string
def getTheElementListForSpecificRegexContent(regexContent, soup):
    returnList= []
    try:
#         print("getTheElementListForSpecificRegexContent() soup(text=re.compile(regexContent)):",soup(text=re.compile(regexContent)))
        for counter in soup(text=re.compile(regexContent)):
#             print("getTheElementListForSpecificRegexContent() counter.parent:",counter.parent)
            returnList.append(counter.parent)
        return returnList
    except Exception as e:
        print(e)
        return returnList

#Find scores for set of element
def defineScoreForElementList(elementList, comparatorValue, highProbailityTags, elementType):
    scoreList = []
    for element in elementList:
        similarityCounterValue = similar(comparatorValue, element.string) #Find similarity
        
        #Add weighted factor if the tag is found
        for tag in highProbailityTags:
            if tag in element.name:
                similarityCounterValue = similarityCounterValue + weightFactorForTheTag
                
        #Add weighted factor if the elementType is found
        if elementType in json.dumps(element.attrs):
            similarityCounterValue = similarityCounterValue + weightFactorForTheTag
            
        scoreList.append(similarityCounterValue)
        
    print("defineScoreForElementList() len(scoreList):", len(scoreList))
    print("defineScoreForElementList() len(elementList):", len(elementList))
    return scoreList

"""
    Find top elements which maps to the comparator: 
        typeOfComparators is set of values which a element's string/content should contain
        highProbailityTags are tags which will likely have those typeOfComparators values so give them more weight
        elementType is useful because the attribute of the tag usually contain such value
"""
def brainOfElementExtractor(typeOfComparators, soup, highProbailityTags, elementType):
    
    elementList = []   
    
    for comparatorValue in typeOfComparators:
        print("brainOfElementExtractor() comparatorValue",comparatorValue)
        elementList = elementList + getTheElementListForSpecificRegexContent(comparatorValue, soup)
        print("brainOfElementExtractor() len(elementList):", len(elementList))
    
#     print(elementList) #For Debugging
#     print(defineScoreForElementList(elementList, comparatorValue, highProbailityTags, elementType)) #For Debugging
    
    #Remove Duplicate Elements from list
    elementListCurated = []
    for i in elementList: 
        if i not in elementListCurated: 
            elementListCurated.append(i)
    
    scoreList = defineScoreForElementList(elementListCurated, comparatorValue, highProbailityTags, elementType)
    print("brainOfElementExtractor() len(scoreList):", len(scoreList))
    
    #Check if list is empty
    if len(scoreList) == 0:
        return None
    
    returnListOfElements = []
    returnListOfScores = []
    if max(scoreList)>highlySimilarValue: #Check if even one value has 'High' similarity score. If yes, then only send those elements
        for i in range(0, len(scoreList)):
            if scoreList[i]>highlySimilarValue:
                returnListOfElements.append(elementListCurated[i])
                returnListOfScores.append(scoreList[i])
                
    elif max(scoreList)>moderatleySimilarValue: #Check if it passes 'Moderately'
        for i in range(0, len(scoreList)):
            if scoreList[i]>moderatelySimilarValue:
                returnListOfElements.append(elementListCurated[i])
                returnListOfScores.append(scoreList[i])
    else:
        for i in range(0, len(scoreList)): #Check if it passes bare minimum
            if scoreList[i]>bareMinimumSimilarValue:
                returnListOfElements.append(elementListCurated[i])
                returnListOfScores.append(scoreList[i])
    
    return returnListOfElements, returnListOfScores

In [29]:
def getLocationForElements(driver, elementList, scoreList, XPathForElements):
    updatedElementsList = [] 
    updatedScoresList = []
    updatedXPathForElements = []
    locationOfElement = []
    sizeOfElement = []
    for i in range(0,len(XPathForElements)):
        try:
            driverElement = driver.find_element_by_xpath(XPathForElements[i])
            updatedElementsList.append(elementList[i])
            updatedScoresList.append(scoreList[i])
            updatedXPathForElements.append(XPathForElements[i])
            locationOfElement.append(driverElement.location)
            sizeOfElement.append(driverElement.size)
        except Exception as e:
            print(e)
            continue
    return updatedElementsList, updatedScoresList, updatedXPathForElements, locationOfElement, sizeOfElement

##### Price

In [13]:
# Price is supposed to be defined in the following tags, so give higher weight
priceHighProbailityTags = ["h", "span"]
weightFactorForTheTag = 0.3

In [14]:
#Try different varaitions of Price Text
#Order: \$379.00, 379.00, \$379, 379
def findPriceElementTagList(gShopPrice, soup): #gShopPrice = $379.00
    
    typeOfPrices = []
    
    gShopPrice = gShopPrice.replace("now","").strip() #GShop sometimes gives prices with now suffix like "$0.00 now"
    
    if "$" in gShopPrice:
        typeOfPrices.append(gShopPrice.replace("$", "\$"))  #gShopPriceUpdated = \$379.00; Required because $ is reserved keyword for regex
        
    typeOfPrices.append(gShopPrice.replace("$","")) #gShopPriceUpdated = 379.00
    
    typeOfPrices.append(gShopPrice.replace("$", "\$").split(".")[0]) #gShopPriceUpdated = \$379
        
    typeOfPrices.append(gShopPrice.replace("$","").split(".")[0]) #gShopPriceUpdated = 379
    
    return typeOfPrices

##### Heading

In [15]:
# Heading is supposed to be defined in the following tags, so give higher weight
headingHighProbailityTags = ["h", "span"]
weightFactorForTheTag = 0.3
minimumNumberOfSpaces = 2

In [16]:
from itertools import combinations

#Try different varaitions of Heading Text
def findHeadingElementTagList(gShopHeading, soup):
    
    typeOfHeadings = []
    
    gShopHeading = gShopHeading.strip()
    noOfSpaces = gShopHeading.count(' ')
    print(noOfSpaces)
    listOfWordsInShopHeading = gShopHeading.split(" ")
    if noOfSpaces>minimumNumberOfSpaces:
        for i in range(3,noOfSpaces+1):
            for j in list(combinations(listOfWordsInShopHeading, i)):
                counterString = ""
                for k in j:
                    counterString = counterString + k + " "
                typeOfHeadings.append(counterString.strip())
                
    typeOfHeadings.append(gShopHeading)
    return typeOfHeadings

### Testing

In [17]:
URL = "https://www.walmart.com/ip/Farberware-3-2-Quart-Digital-Oil-Less-Fryer-White/264698854?athcpid=264698854&athpgid=athenaHomepage&athcgid=null&athznid=BestInDeals&athieid=v1&athstid=CS020&athguid=466001f5-46cfa622-5eb821569a18a716&athancid=null&athena=true"

In [18]:
soup = getHTML(URL)

Built Soup


##### Price

In [19]:
gShopPrice = "$39.00"

In [51]:
typeOfPrices = findPriceElementTagList(gShopPrice, soup)
print(typeOfPrices)
priceElements, priceScores = brainOfElementExtractor(typeOfPrices, soup, priceHighProbailityTags, "price")
print(priceElements)
print(priceScores)
XPathForPriceElements = getListOfXPath(priceElements)

['\\$39.00', '39.00', '\\$39', '39']
brainOfElementExtractor() comparatorValue \$39.00
brainOfElementExtractor() len(elementList): 2
brainOfElementExtractor() comparatorValue 39.00
brainOfElementExtractor() len(elementList): 4
brainOfElementExtractor() comparatorValue \$39
brainOfElementExtractor() len(elementList): 6
brainOfElementExtractor() comparatorValue 39
brainOfElementExtractor() len(elementList): 14
defineScoreForElementList() len(scoreList): 7
defineScoreForElementList() len(elementList): 7
brainOfElementExtractor() len(scoreList): 7
[<span class="visuallyhidden">$39.00</span>, <span class="price-characteristic" content="39.00" itemprop="price">39</span>, <span class="price-characteristic">39</span>]
[0.8, 1.6, 1.6]
{'class': ['price', 'display-inline-block', 'arrange-fit', 'price', 'price--stylized']}
{'class': ['hide-content', 'display-inline-block-m']}
{'class': ['prod-PriceHero']}
{'class': ['display-inline-block-xs', 'prod-PaddingRight--xs', 'valign-top'], 'id': 'price'}

In [21]:
XPathForPriceElements

["//span[@id='price']/div/span/span/span",
 "//span[@id='price']/div/span/span/span/span",
 "//span[@id='price']/div/span/span/span/span"]

In [52]:
print(priceElements)
print(priceScores)

[<span class="visuallyhidden">$39.00</span>, <span class="price-characteristic" content="39.00" itemprop="price">39</span>, <span class="price-characteristic">39</span>]
[0.8, 1.6, 1.6]


In [ ]:
<span class="price-characteristic">39</span>
1. To find similarity: 1
2. Find if the tag is heading or span: 0.3
3. Find if the tag's attributes have "price": 0.3

##### Heading

In [37]:
gShopHeading = "Product Highlights"

In [38]:
# gShopHeading = gShopHeading.translate ({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
# gShopHeading.split(" ")

In [39]:
typeOfHeadings = findHeadingElementTagList(gShopHeading, soup)
headingElements, headingScores = brainOfElementExtractor(typeOfHeadings, soup, headingHighProbailityTags, "heading")
XPathForHeadingElements = getListOfXPath(headingElements)

1
brainOfElementExtractor() comparatorValue Product Highlights
brainOfElementExtractor() len(elementList): 1
defineScoreForElementList() len(scoreList): 1
defineScoreForElementList() len(elementList): 1
brainOfElementExtractor() len(scoreList): 1
{'class': ['prod-ProductHighlights-content'], 'data-tl-id': 'ProductPage-ProductHighlights'}
{'id': 'product-highlights', 'tabindex': '0', 'class': ['prod-ProductHighlights', 'prod-ProductHighlightsSection', 'xxs-padding-sides']}
true


In [40]:
XPathForHeadingElements

["//div[@id='product-highlights']/div/h3"]

In [32]:
# /html/body/div[1]/div/div/div[2]/div/div[1]/div/div[1]/div/div/div/div/div[3]/div[5]/div/div[3]/div/h1

##### Locality

In [27]:
#Get Selenium Object for that XPath
driver2 = getSeleniumDriverForChrome(googleChomreDriverPath, False)
driver2.get(URL)
# driver2.implicitly_wait(10)
driver2.maximize_window()

In [35]:
updatedPriceElements, updatedPriceScores, updatedXPathForPriceElements, locationOfPriceElement, sizeOfPriceElement = getLocationForElements(driver2, priceElements, priceScores, XPathForPriceElements)

In [41]:
updatedHeadingElements, updatedHeadingScores, updatedXPathForHeadingElements, locationOfHeadingElement, sizeOfHeadingElement = getLocationForElements(driver2, headingElements, headingScores, XPathForHeadingElements)

In [43]:
locationOfPriceElement

[{'x': 759, 'y': 354}, {'x': 760, 'y': 356}, {'x': 760, 'y': 356}]

In [54]:
sizeOfPriceElement

[{'height': 1, 'width': 1},
 {'height': 34, 'width': 12},
 {'height': 34, 'width': 12}]

In [44]:
locationOfHeadingElement

[{'x': 760, 'y': 824}]

In [46]:
import math  
def calculateDistance(x1,y1,x2,y2):  
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
    return dist 

In [47]:
for price in locationOfPriceElement:
    for heading in locationOfHeadingElement:
        print(calculateDistance(price['x'],price['y'],heading['x'],heading['y']))

470.00106382858326
468.0
468.0


In [48]:
updatedPriceElements

[<span class="visuallyhidden">$39.00</span>,
 <span class="price-characteristic" content="39.00" itemprop="price">39</span>,
 <span class="price-characteristic">39</span>]

In [49]:
updatedHeadingElements

[<h3 class="prod-ProductHighlights-header font-normal">Product Highlights</h3>]

In [33]:
# updatedPriceElements = [] 
# updatedPriceScores = []
# updatedXPathForPriceElements = []
# locationOfPriceElement = []
# sizeOfElement = []
# for i in range(0,len(XPathForPriceElements)):
#     try:
#         driverElement = driver2.find_element_by_xpath(XPathForPriceElements[i])
#         updatedPriceElements.append(priceElements[i])
#         updatedPriceScores.append(priceScores[i])
#         updatedXPathForPriceElements.append(XPathForPriceElements[i])
#         locationOfElement.append(driverElement.location)
#         sizeOfElement.append(driverElement.size)
#     except Exception as e:
#         print(e)
#         continue

name 'locationOfElement' is not defined
name 'locationOfElement' is not defined
name 'locationOfElement' is not defined


In [36]:
sizeOfPriceElement

[{'height': 1, 'width': 1},
 {'height': 34, 'width': 12},
 {'height': 34, 'width': 12}]

In [ ]:
for 

In [113]:
driverElement = driver2.find_element_by_xpath(getMyXPath(priceElements[2])) #Find XPath
print(driverElement)
print("driverElement.location_once_scrolled_into_view :",driverElement.location_once_scrolled_into_view )
print("driverElement.location :",driverElement.location)
print("driverElement.size :",driverElement.size)

{'class': ['price-group'], 'aria-hidden': 'true'}
{'class': ['price', 'display-inline-block', 'arrange-fit', 'price']}
{'class': ['hide-content-m']}
{'class': ['prod-PriceHero']}
{'class': ['display-inline-block-xs', 'prod-PaddingRight--xs', 'valign-top'], 'id': 'price'}
true
<selenium.webdriver.remote.webelement.WebElement (session="669be2320d619fdec5362434401d8422", element="12e6d7b8-2bcd-4f2e-9ec8-13ca49df92e5")>
driverElement.location_once_scrolled_into_view : {'x': 760, 'y': 1}
driverElement.location : {'x': 760, 'y': 354}
driverElement.size : {'height': 38, 'width': 16}


In [114]:
driver2.execute_script("arguments[0].setAttribute('style', 'border: 2px solid #73AD21')", driverElement);

In [42]:
driver2.quit()

In [74]:
def getTheTagElementListForPrice(gShopPriceUpdated, soup):
    
    returnPriceElementTag = None
    
    try:
        dummyVar = soup(text=re.compile(gShopPriceUpdated))
#         print(dummyVar)
        for elem in dummyVar:
            
            #find similarity
#             if similar(gShopPrice, headingTag.string)

#             print("elem: ",elem)
#             print("elem.parent: ",elem.parent)
#             print("elem.parent.name: ",elem.parent.name)
            if returnPriceElementTag == None: #The first element is the return value unless we encounter a heading tag
                returnPriceElementTag = elem.parent
            if "h" in elem.parent.name: #Found the heading tag, so return this tag and break the loop
                returnPriceElementTag = elem.parent
                return returnPriceElementTag
            if "span" in elem.parent.name: #Found the span tag, so return this tag and break the loop
                print("elem: ",elem)
                print("elem.parent: ",elem.parent)
                print("elem.parent.name: ",elem.parent.name)
                print(str(elem.parent))
                print("" + json.dumps(elem.parent.attrs))
                returnPriceElementTag = elem.parent
                return returnPriceElementTag
    except Exception as e:
        print("Error in getTheTagElementForPrice(gShopPriceUpdated, soup)")
    return returnPriceElementTag

#Try different varaitions of Price Text
#Order: \$379.00, 379.00, \$379, 379
def findPriceElementTag(gShopPrice, soup): #gShopPrice = $379.00
    
    gShopPrice = gShopPrice.replace("now","").strip() #GShop soemtimes gives prices with now suffix like "$0.00 now"
    print("Hi",gShopPrice)
    
    if "$" in gShopPrice:
        gShopPriceUpdated = gShopPrice.replace("$", "\$") #gShopPriceUpdated = \$379.00; Required because $ is reserved keyword for regex
        print(gShopPriceUpdated)
        returnPriceElementTag = getTheTagElementListForPrice(gShopPriceUpdated, soup)
        if returnPriceElementTag != None and len(str(returnPriceElementTag))<400:
            return returnPriceElementTag
    
    gShopPriceUpdated = gShopPrice.replace("$","") #gShopPriceUpdated = 379.00
    print(gShopPriceUpdated)
    returnPriceElementTag = getTheTagElementListForPrice(gShopPriceUpdated, soup)
    if returnPriceElementTag != None and len(str(returnPriceElementTag))<400:
        return returnPriceElementTag
        
    gShopPriceUpdated = gShopPrice.replace("$", "\$").split(".")[0] #gShopPriceUpdated = \$379
    print(gShopPriceUpdated)
    returnPriceElementTag = getTheTagElementListForPrice(gShopPriceUpdated, soup)
    if returnPriceElementTag != None and len(str(returnPriceElementTag))<400:
        return returnPriceElementTag
        
    gShopPriceUpdated = gShopPrice.replace("$","").split(".")[0] #gShopPriceUpdated = 379
    print(gShopPriceUpdated)
    returnPriceElementTag = getTheTagElementListForPrice(gShopPriceUpdated, soup)
    if returnPriceElementTag != None and len(str(returnPriceElementTag))<400:
        return returnPriceElementTag
        
    return None

In [88]:
returnPriceElementTag = findPriceElementTag(priceOfCurrentScreenshot, soup)

Hi 39
39
elem:  $39.00
elem.parent:  <span class="visuallyhidden">$39.00</span>
elem.parent.name:  span
<span class="visuallyhidden">$39.00</span>
{"class": ["visuallyhidden"]}


In [89]:
getMyStackXPath(returnPriceElementTag)

xpath_tag: span
xpath_tag.attrs: {'class': ['visuallyhidden']}
xpath_index: 1
components: ['span']
xpath_tag: span
xpath_tag.attrs: {'class': ['price', 'display-inline-block', 'arrange-fit', 'price', 'price--stylized']}
xpath_index: 1
components: ['span', 'span']
xpath_tag: span
xpath_tag.attrs: {'class': ['hide-content', 'display-inline-block-m']}
xpath_index: 1
components: ['span', 'span', 'span']
xpath_tag: div
xpath_tag.attrs: {'class': ['prod-PriceHero']}
xpath_index: 1
components: ['span', 'span', 'span', 'div']
true


"/span[@id='price']/div/span/span/span"